# Clasificación de MNIST con un Perceptrón Multicapa (MLP)

## Perceptrón Multicapa (MLP)

Un **Perceptrón Multicapa (MLP)** es una red neuronal artificial compuesta por varias capas de neuronas. Es uno de los modelos más básicos y clásicos de redes neuronales, utilizado para tareas de clasificación y regresión.

###  Estructura del MLP:

1. **Capa de entrada (Input layer):**
   - Recibe los datos de entrada.
   - En el caso del dataset MNIST, cada imagen tiene tamaño 28x28 píxeles, que se convierte en un vector de 784 elementos.
   - Cada elemento representa un píxel con un valor entre 0 y 1 (después de normalizar).

2. **Capas ocultas (Hidden layers):**
   - Compuestas por neuronas densamente conectadas (fully connected).
   - Usan funciones de activación no lineales (como ReLU).
   - Permiten aprender representaciones internas complejas.

3. **Capa de salida (Output layer):**
   - Tiene 10 neuronas, una por cada clase (dígitos del 0 al 9).
   - Utiliza activación **softmax** para generar una distribución de probabilidad.


## Dataset MNIST?

El dataset MNIST contiene:
- 60,000 imágenes de entrenamiento y 10,000 de prueba.
- Cada imagen es de 28x28 píxeles en escala de grises.
- Las etiquetas son dígitos del 0 al 9.



## Entrenamiento de un MLP

El entrenamiento sigue un ciclo de pasos iterativos para ajustar los pesos de la red y minimizar el error en las predicciones.

### 1. **Propagación hacia adelante (Forward pass)**

Se calculan las salidas de cada capa de la red usando los pesos actuales:

$$
\text{z}^{(1)} = W^{(1)}x + b^{(1)} \quad \text{→ entrada a la capa oculta}
$$

$$
\text{a}^{(1)} = \text{ReLU}(\text{z}^{(1)}) \quad \text{→ salida de la capa oculta}
$$

$$
\text{z}^{(2)} = W^{(2)}a^{(1)} + b^{(2)} \quad \text{→ entrada a la capa de salida}
$$

$$
\hat{y} = \text{softmax}(z^{(2)}) \quad \text{→ probabilidades para cada clase}
$$

### 2. **Cálculo de la pérdida (Loss function)**

Se compara la predicción del modelo con la etiqueta real mediante una función de pérdida. En clasificación multiclase, se usa la **entropía cruzada (cross-entropy)**:

$$
\mathcal{L} = -\sum_{i=1}^{10} y_i \log(\hat{y}_i)
$$

Donde:
- $ y_i $ es la etiqueta real (en one-hot encoding).
- $ \hat{y}_i $ es la probabilidad predicha para la clase $i $.

### 3. **Retropropagación y ajuste de pesos (Backpropagation + Gradient Descent)**

Después de calcular la pérdida, necesitamos ajustar los pesos para reducir el error. Esto se hace en varios pasos:

#### a) Error en la capa de salida

El error que tiene la red en la salida es la diferencia entre la predicción y la etiqueta real:

$$
\delta^{(2)} = \hat{y} - y
$$

Aquí:
- $\delta^{(2)}$ es el vector de errores para la capa de salida.
- $\hat{y}$ es la salida predicha para cada clase.
- $y$ es la etiqueta verdadera.

#### b) Error en la capa oculta

Este error se propaga hacia atrás para saber cuánto influyó cada neurona oculta en el error final:

$$
\delta^{(1)} = (W^{(2)})^T \delta^{(2)} \circ \text{ReLU}'(z^{(1)})
$$

Donde:
- $(W^{(2)})^T$ es la matriz de pesos de la capa de salida transpuesta.
- $\circ$ indica multiplicación elemento a elemento.
- $\text{ReLU}'(z^{(1)})$ es la derivada de ReLU, que vale 1 si $z^{(1)} > 0$, y 0 si no.

#### c) Cálculo de gradientes para pesos y sesgos

Con los errores calculados, ahora determinamos cuánto cambiar cada peso y sesgo para mejorar la predicción.

- Gradiente para pesos de la capa de salida:

$$
\frac{\partial \mathcal{L}}{\partial W^{(2)}} = \delta^{(2)} (a^{(1)})^T
$$

- Gradiente para sesgos de la capa de salida:

$$
\frac{\partial \mathcal{L}}{\partial b^{(2)}} = \delta^{(2)}
$$

- Gradiente para pesos de la capa oculta:

$$
\frac{\partial \mathcal{L}}{\partial W^{(1)}} = \delta^{(1)} (x)^T
$$

- Gradiente para sesgos de la capa oculta:

$$
\frac{\partial \mathcal{L}}{\partial b^{(1)}} = \delta^{(1)}
$$

#### d) Actualización de pesos y sesgos

Finalmente, ajustamos los pesos y sesgos usando la tasa de aprendizaje $\eta$:

$$
W^{(l)} := W^{(l)} - \eta \frac{\partial \mathcal{L}}{\partial W^{(l)}}
$$

$$
b^{(l)} := b^{(l)} - \eta \frac{\partial \mathcal{L}}{\partial b^{(l)}}
$$

con $l = 1, 2$ indicando la capa (oculta o salida).cto a ese peso.


###  Entrenamiento por lotes y épocas

- El conjunto de datos se divide en lotes (batches).
- Cada lote se usa para actualizar los pesos una vez.
- Una **época** es un recorrido completo por todo el conjunto de entrenamiento.
- El modelo mejora iterativamente al ver más datos.



###  ¿Qué aprende el modelo?

- Comienza con pesos aleatorios.
- A través del entrenamiento, ajusta los pesos para minimizar el error.
- Aprende a asociar patrones visuales (píxeles) con etiquetas (números).
- El objetivo es **generalizar bien** para predecir correctamente imágenes no vistas.


Este proceso permite que un MLP aprenda a clasificar imágenes de dígitos escritos a mano con alta precisión.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Cargar datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar píxeles
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

#  Learning rate
learning_rate_opt = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)

#  Modelo MLP con 124 neuronas (óptimo)
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compilar con optimizador modificado
model.compile(
    optimizer=optimizer,  # <-- Optimizer con tasa de aprendizaje óptima
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Entrenar modelo
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    verbose=2
)

# Graficar pérdida
plt.plot(history.history['loss'], label='Pérdida entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida validación')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Convergencia de la función de pérdida')
plt.show()

# Evaluar en test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

# Predicciones
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Reporte sklearn
print("\nReporte de clasificación (precision, recall, f1-score):")
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)


# Diseño de Experimentos para un Perceptrón Multicapa (MLP)

## 1. Define el objetivo del experimento

- Optimizar la arquitectura (número de capas, neuronas).
- Optimizar hiperparámetros de entrenamiento (learning rate, batch size, regularización).
- Comparar funciones de activación o tipos de optimizadores.

## 2. Tipos de factores

- **Continuos:** learning rate, momentum, dropout rate, número de neuronas (discreto con muchos niveles).
- **Discretos:** número de capas, funciones de activación, optimizadores.
- **Categóricos:** tipos de regularización, funciones de activación, optimizador.

## 3. Diseños experimentales comunes para MLP

| Método                        | Descripción                                               | Ventajas                                | Limitaciones                         |
|-------------------------------|-----------------------------------------------------------|----------------------------------------|-------------------------------------|
| **Búsqueda en malla (Grid Search)**        | Probar combinaciones en rejilla regular.                 | Fácil de implementar, sistemático.    | Costoso, explora pobremente el espacio. |
| **Búsqueda aleatoria (Random Search)**     | Probar puntos aleatorios en el espacio.                   | Más eficiente que grid, fácil.         | No garantiza cobertura completa.   |
| **Optimización bayesiana**                  | Modela la función objetivo con un modelo probabilístico. | Muy eficiente, menos ejecuciones.      | Más compleja, requiere librerías.   |
| **Diseño de Superficie de Respuesta (RSM)**| Usa diseños factoriales y centrales para ajustar modelo. | Balance entre exploración y modelado.  | Requiere función respuesta suave, pocos factores. |
| **Diseño factorial fraccional**             | Explora combinaciones seleccionadas para reducir pruebas.| Reduce experimentos, explora interacciones principales. | Menos exhaustivo, puede perder interacciones. |
| **Algoritmos heurísticos**             | Son una combinación entre exploración y explotación del espacio.| Puede explorar mas parte del espacio. | En ocasiones puede ser costoso computacionalmente. |

## 4. Recomendaciones prácticas

- Para pocos factores continuos: **RSM con diseño Central Compuesto (CCD)** para entender efectos e interacciones.
- Para muchos hiperparámetros o combinaciones: **búsqueda aleatoria o bayesiana**.
- Usar siempre conjunto de validación para medir desempeño.
- Realizar réplicas en condiciones centrales para estimar variabilidad.

## 5. Variables recomendadas a experimentar en un MLP

- Learning rate (continuo)
- Número de neuronas por capa (discreto)
- Número de capas (discreto)
- Batch size (discreto)
- Regularización (L2 lambda, dropout rate)
- Función de activación (ReLU, tanh, sigmoid)

## 6. Ejemplo óptimo para un MLP simple

- Para optimizar learning rate, neuronas y batch size: **Diseño RSM** es ideal.
- Para probar arquitecturas y optimizadores: combinar diseño factorial fraccional (variables categóricas) y RSM o búsqueda aleatoria (variables continuas).



## Resumen: ¿Cuál es la mejor forma?

- **Pocos factores numéricos:** Diseño Central Compuesto (CCD) con RSM.
- **Muchos hiperparámetros:** Optimización bayesiana o búsqueda aleatoria.
- **Experimentos rápidos:** Grid search con validación cruzada.


# Diseño Central Compuesto (CCD) y Método de Superficie de Respuesta (RSM)

## Introducción al Método de Superficie de Respuesta (RSM)

El Método de Superficie de Respuesta (Response Surface Methodology, RSM) es un conjunto de técnicas estadísticas para modelar y analizar problemas en los cuales una o más variables de respuesta $y$ dependen de varias variables independientes o factores $x_1, x_2, ..., x_k$.

El objetivo principal del RSM es:

- Encontrar las condiciones óptimas de los factores que maximicen o minimicen la respuesta.
- Modelar la relación entre las variables de entrada y la respuesta mediante funciones matemáticas simples, generalmente polinomios de segundo grado (cuadráticos).


##  Diseño Central Compuesto (CCD)

El Diseño Central Compuesto (Central Composite Design, CCD) es uno de los diseños experimentales más usados para ajustar modelos de superficie de respuesta de segundo orden (cuadráticos). Es ideal cuando se tienen pocos factores numéricos y se desea explorar no solo los efectos lineales sino también los efectos cuadráticos y las interacciones entre factores.

### Componentes del CCD:

- **Puntos factoriales:** Combinaciones de los niveles altos (+1) y bajos (-1) para cada factor, formando un diseño factorial completo ($2^k$ puntos para $k$ factores).
- **Puntos axiales (puntos estrella):** Extienden el rango de exploración fuera del diseño factorial con niveles a distancia $\pm \alpha$ en cada factor, manteniendo los demás en nivel central (0). Estos puntos permiten estimar efectos cuadráticos.
- **Puntos centrales:** Uno o más puntos en el centro del diseño (nivel 0 para todos los factores) que ayudan a estimar la variabilidad experimental y la curvatura.

### Parámetro $\alpha$:

El valor de $\alpha$ define la distancia de los puntos axiales al centro y puede elegirse para mantener propiedades estadísticas específicas, como la rotabilidad (igual precisión en todas las direcciones).



## Codificación de factores

Para facilitar el análisis, los niveles de los factores se codifican de forma estandarizada:

$$
x_i = \frac{\text{valor real} - \text{valor central}}{\text{semi-rango}}
$$
donde:

- $x_i$ es el nivel codificado del factor $i$, típicamente en el rango $[-1, +1]$ para los puntos factoriales.
- El valor central es el punto medio del rango real del factor.
- El semi-rango es la mitad del rango real.

Esta codificación permite comparar efectos de diferentes factores en una escala común.



##  Ajuste del modelo de superficie de respuesta

Con los datos obtenidos del experimento CCD, se ajusta un modelo polinomial cuadrático de la forma:

$$
y = \beta_0 + \sum_{i=1}^k \beta_i x_i + \sum_{i=1}^k \beta_{ii} x_i^2 + \sum_{i<j} \beta_{ij} x_i x_j + \varepsilon
$$

donde:

- $y$ es la variable respuesta (p. ej., precisión del modelo MLP).
- $\beta_0$ es la ordenada al origen.
- $\beta_i$, $\beta_{ii}$, y $\beta_{ij}$ son los coeficientes que representan los efectos lineales, cuadráticos e interacción, respectivamente.
- $\varepsilon$ es el término de error experimental.

Este modelo permite:

- Identificar los efectos significativos de cada factor y sus interacciones.
- Visualizar la superficie de respuesta.
- Encontrar los valores óptimos de los factores que maximizan o minimizan $y$.


##  Aplicación práctica en MLP

En el contexto de un Perceptrón Multicapa (MLP):

- Los factores pueden ser hiperparámetros como tasa de aprendizaje, número de neuronas, número de capas, etc.
- La respuesta puede ser la precisión, error de validación o cualquier métrica de desempeño.
- Utilizando CCD y RSM, podemos diseñar un conjunto reducido pero eficiente de experimentos para explorar cómo los hiperparámetros afectan el desempeño del MLP y encontrar configuraciones óptimas sin realizar búsquedas exhaustivas.




# Optimización de un Perceptrón Multicapa (MLP) con Superficie de Respuesta (RSM) y Diseño Central Compuesto (CCD)

##  Objetivo

Optimizar el rendimiento de un Perceptrón Multicapa (MLP) ajustando dos hiperparámetros clave:

- **Tasa de aprendizaje** (`learning_rate`)
- **Número de neuronas ocultas** (`units`)

Usamos el **Método de Superficie de Respuesta (RSM)** con un **Diseño Central Compuesto (CCD)** para construir un modelo cuadrático que relacione estos hiperparámetros con el desempeño del MLP (precisión de validación).


##  Paso 1: Selección de factores y niveles

Seleccionamos dos factores, cada uno con 5 niveles: codificados como $- \alpha, -1, 0, +1, +\alpha$, donde $\alpha = \sqrt{2} \approx 1.414$ (valor común para dos factores).

###  Factores y niveles reales

| Factor              | $-\alpha$  | $-1$      | $0$       | $+1$      | $+\alpha$  |
|---------------------|------------|-----------|-----------|-----------|------------|
| Learning rate       | 0.00001    | 0.0001    | 0.001     | 0.01      | 0.1        |
| Neuronas ocultas    | 32         | 64        | 128       | 256       | 512        |

###  Puntos del diseño CCD

El diseño CCD incluye:

- **4 puntos factoriales**: combinaciones de niveles $-1$ y $+1$
- **4 puntos axiales**: niveles extremos $- \alpha$ y $+ \alpha$ con los demás factores en 0
- **5 réplicas del punto central**: todos los factores en nivel 0

Esto da un total de **13 experimentos**.

| Punto | $x_1$ (codificado) | $x_2$ (codificado) | learning\_rate | units |
|-------|--------------------|--------------------|----------------|--------|
| F1    | -1                 | -1                 | 0.0001         | 64     |
| F2    | +1                 | -1                 | 0.01           | 64     |
| F3    | -1                 | +1                 | 0.0001         | 256    |
| F4    | +1                 | +1                 | 0.01           | 256    |
| A1    | -1.414             | 0                  | 0.00004        | 128    |
| A2    | +1.414             | 0                  | 0.0252         | 128    |
| A3    | 0                  | -1.414             | 0.001          | 45     |
| A4    | 0                  | +1.414             | 0.001          | 362    |
| C1    | 0                  | 0                  | 0.001          | 128    |
| C2    | 0                  | 0                  | 0.001          | 128    |
| C3    | 0                  | 0                  | 0.001          | 128    |
| C4    | 0                  | 0                  | 0.001          | 128    |
| C5    | 0                  | 0                  | 0.001          | 128    |

##  Paso 2: Modelo cuadrático de respuesta

Usamos un modelo cuadrático de segundo orden:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_{11} x_1^2 + \beta_{22} x_2^2 + \beta_{12} x_1 x_2 + \varepsilon
$$

Donde:

- $y$ es la precisión de validación del MLP
- $x_1$: nivel codificado del `learning_rate`
- $x_2$: nivel codificado del número de `units`
- $\beta$ son los coeficientes a estimar
- $\varepsilon$ es el error aleatorio

Este modelo permite capturar efectos lineales, cuadráticos y de interacción entre los hiperparámetros.


## Paso 3: Ejecución del experimento

1. **Construir la matriz de diseño CCD** con los niveles codificados ($x_1$, $x_2$).
2. **Convertir los niveles codificados** a valores reales de hiperparámetros.
3. **Entrenar el MLP** con cada combinación y registrar la precisión de validación.
4. **Ajustar el modelo cuadrático** usando regresión lineal (con `statsmodels` o `sklearn.linear_model.LinearRegression`).
5. **Analizar los resultados**:
   - Significancia de los términos (lineales, cuadráticos, interacción)
   - Superficie de respuesta (mapa 3D o contornos)
   - Hiperparámetros óptimos


## Ventajas del enfoque

- Eficiencia: Menos combinaciones que una búsqueda en malla (grid search).
- Modelo explicativo y predictivo del desempeño del MLP.
- Posibilidad de visualizar e interpretar la superficie de respuesta.
- Permite evaluar **efectos individuales y combinados** de los hiperparámetros.


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

# Cargar y normalizar los datos MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# Dividir entrenamiento en entrenamiento + validación
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# -----------------------------
# Función de entrenamiento
# -----------------------------
def train_model(learning_rate, units):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units, activation='relu'),
        Dense(10, activation='softmax')
    ])

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x_train, y_train,
        epochs=10,
        batch_size=64,
        validation_data=(x_val, y_val),
        verbose=0
    )

    # Devolver última precisión de validación
    return history.history['val_accuracy'][-1]

# -----------------------------
# Tabla del diseño CCD
# -----------------------------
design = pd.DataFrame([
    [-1, -1], [1, -1], [-1, 1], [1, 1],     # factorial
    [-1.414, 0], [1.414, 0], [0, -1.414], [0, 1.414],  # axial
    [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]  # centro (5 rep)
], columns=["x1", "x2"])

# Conversión a hiperparámetros reales
def coded_to_real(x1, x2):
    # learning_rate en escala log10
    learning_rate = 0.001 * (10 ** x1)
    units = int(round(128 * (2 ** x2)))
    return learning_rate, units

# -----------------------------
# Entrenamiento en cada punto
# -----------------------------
results = []

for i, row in design.iterrows():
    x1, x2 = row["x1"], row["x2"]
    lr, u = coded_to_real(x1, x2)

    print(f"Ejecutando experimento {i+1}: lr={lr:.5f}, units={u}")
    acc = train_model(lr, u)

    results.append({
        "x1": x1,
        "x2": x2,
        "learning_rate": lr,
        "units": u,
        "val_accuracy": acc
    })

# Convertir a DataFrame
results_df = pd.DataFrame(results)
print(results_df)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Ejecutando experimento 1: lr=0.00010, units=64


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Ejecutando experimento 2: lr=0.01000, units=64
Ejecutando experimento 3: lr=0.00010, units=256
Ejecutando experimento 4: lr=0.01000, units=256
Ejecutando experimento 5: lr=0.00004, units=128
Ejecutando experimento 6: lr=0.02594, units=128
Ejecutando experimento 7: lr=0.00100, units=48
Ejecutando experimento 8: lr=0.00100, units=341
Ejecutando experimento 9: lr=0.00100, units=128
Ejecutando experimento 10: lr=0.00100, units=128
Ejecutando experimento 11: lr=0.00100, units=128
Ejecutando experimento 12: lr=0.00100, units=128
Ejecutando experimento 13: lr=0.00100, units=128
       x1     x2  learning_rate  units  val_accuracy
0  -1.000 -1.000       0.000100     64      0.940417
1   1.000 -1.000       0.010000     64      0.966667
2  -1.000  1.000       0.000100    256      0.960917
3   1.000  1.000       0.010000    256      0.967417
4  -1.414  0.000       0.000039    128      0.929833
5   1.414  0.000       0.025942    128      0.943750
6   0.000 -1.414       0.001000     48      0.96675

## Paso 4: Ajuste del modelo cuadrático de segundo orden
Una vez entrenado el perceptrón para cada combinación de hiperparámetros del diseño CCD y registrada la precisión de validación, queremos ajustar un **modelo de segundo orden** que relacione los factores con la respuesta.

El modelo que queremos ajustar es:

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_{11} x_1^2 + \beta_{22} x_2^2 + \beta_{12} x_1 x_2 + \varepsilon
$$

donde:

- $y$ es la **precisión de validación**.
- $x_1$ es el **nivel codificado del learning rate**.
- $x_2$ es el **nivel codificado del número de unidades**.
- Los $\beta$ son los coeficientes que se estimarán.
- $\varepsilon$ es el error aleatorio (ruido experimental).

- **$\beta_0$**: intercepto (respuesta en el punto central).
- **$\beta_1$ y $\beta_2$**: efectos lineales de los factores.
- **$\beta_{11}$ y $\beta_{22}$**: efectos cuadráticos (curvatura) de cada factor.
- **$\beta_{12}$**: interacción entre factores (cómo cambia el efecto de un factor cuando el otro también cambia).


###  Procedimiento :

1. **Codificación de niveles**:
   - Convertimos cada combinación real de hiperparámetros (learning rate, unidades) a sus niveles codificados: $x_1, x_2$.
   - Esto se hace con la fórmula:

   $$
   x_i = \frac{z_i - z_{i,0}}{\Delta z_i}
   $$

   donde $z_i$ es el valor real del factor, $z_{i,0}$ es el valor central, y $\Delta z_i$ es el paso entre niveles (diferencia entre centro y alto o centro y bajo).

2. **Creación de variables del modelo**:
   - A partir de los niveles codificados, construimos las variables del modelo: $x_1$, $x_2$, $x_1^2$, $x_2^2$, y $x_1x_2$.

3. **Ajuste con regresión**:
   - Usamos el paquete `statsmodels` para ajustar un modelo de regresión lineal múltiple con estos términos como predictores y la precisión como variable respuesta.

4. **Evaluación del modelo**:
   - Obtenemos los coeficientes estimados $\hat{\beta}$.
   - Evaluamos la significancia estadística (p-valores) y el coeficiente de determinación ($R^2$) para ver qué tan bien ajusta el modelo.

5. **Uso del modelo**:
   - El modelo se puede usar para:
     - Predecir la precisión esperada dados nuevos valores de hiperparámetros (dentro del rango).
     - Visualizar la **superficie de respuesta**.
     - Encontrar la combinación óptima que maximiza la precisión.



In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from itertools import product
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist

# 1. Definir niveles reales y codificados
center = {'lr': 0.001, 'units': 128}
step = {'lr': 0.0009, 'units': 64}  # (alto - centro)
alpha = 1.414  # valor axial

# Niveles codificados para CCD
ccd_points = [
    [-1, -1], [-1,  1], [1, -1], [1,  1],  # factorial
    [-alpha, 0], [alpha, 0], [0, -alpha], [0, alpha],  # axiales
    [0, 0], [0, 0], [0, 0]  # 3 repeticiones del centro
]

# Convertimos a hiperparámetros reales
def decode_level(lr_lvl, unit_lvl):
    lr_real = center['lr'] + lr_lvl * step['lr']
    units_real = int(center['units'] + unit_lvl * step['units'])
    return lr_real, units_real

real_hyperparams = [decode_level(lr, u) for lr, u in ccd_points]

# 2. Cargar y normalizar MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_val = x_train[-12000:]
y_val = y_train[-12000:]
x_train = x_train[:-12000]
y_train = y_train[:-12000]

# 3. Entrenar modelo para cada combinación
accuracies = []
for i, (lr, units) in enumerate(real_hyperparams):
    print(f"🔁 Iteración {i+1}: learning_rate={lr}, units={units}")
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    model.fit(x_train, y_train, epochs=5, batch_size=64, verbose=0)
    loss, acc = model.evaluate(x_val, y_val, verbose=0)
    accuracies.append(acc)

# 4. Construir variables del modelo cuadrático
design_df = pd.DataFrame(ccd_points, columns=['x1', 'x2'])  # niveles codificados
design_df['x1^2'] = design_df['x1'] ** 2
design_df['x2^2'] = design_df['x2'] ** 2
design_df['x1*x2'] = design_df['x1'] * design_df['x2']
design_df['accuracy'] = accuracies

# 5. Ajustar modelo con statsmodels
X = design_df[['x1', 'x2', 'x1^2', 'x2^2', 'x1*x2']]
X = sm.add_constant(X)
y = design_df['accuracy']
model = sm.OLS(y, X).fit()
print(model.summary())

ModuleNotFoundError: No module named 'tensorflow'

### Evaluación global del modelo

| Métrica               | Valor      | Interpretación |
|------------------------|------------|----------------|
| **R-squared**          | 0.628      | El modelo explica el **62.8% de la variabilidad** observada en la precisión. Es aceptable pero no excelente. |
| **Adj. R-squared**     | 0.255      | Al ajustar por el número de predictores, la varianza explicada baja a **25.5%**, indicando que algunos términos podrían **no contribuir significativamente**. |
| **F-statistic**        | 1.685      | La prueba F evalúa si el modelo completo es mejor que uno sin predictores. |
| **Prob(F-statistic)**  | 0.291      | No significativo (p > 0.05). **No hay evidencia estadística de que el modelo en conjunto sea útil**. |
| **N° Observaciones**   | 11         | El tamaño muestral es pequeño, lo que puede reducir la precisión de los coeficientes estimados. |


### Interpretación de coeficientes

El modelo ajustado fue:

$$
\hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_{11} x_1^2 + \beta_{22} x_2^2 + \beta_{12} x_1 x_2
$$

| Término     | Coef. | p-valor | Interpretación |
|-------------|-------|---------|----------------|
| **const**   | 0.971 | 0.001   | Precisión esperada en el punto central del diseño. Muy significativa. |
| **$x_1$** (lineal) | 0.161 | 0.097   | Efecto lineal positivo del learning rate. Aunque no es significativo (p > 0.05), es el más cercano a serlo. |
| **$x_2$** (lineal) | 0.0058 | 0.944  | El número de neuronas ocultas **no tiene efecto lineal significativo**. |
| **$x_1^2$** (cuadrático) | -0.165 | 0.138  | Indica curvatura descendente: puede existir un máximo de precisión para un valor intermedio de $x_1$, pero no es concluyente. |
| **$x_2^2$** (cuadrático) | 0.0475 | 0.634  | No hay evidencia de curvatura respecto al número de neuronas ocultas. |
| **$x_1 \cdot x_2$** (interacción) | -0.0034 | 0.977 | La interacción entre factores es prácticamente nula. |


### Conclusiones

- **El único efecto con cierto impacto es el término lineal de $x_1$ (learning rate)**, pero no alcanza significancia estadística.
- **El número de neuronas ocultas no parece influir significativamente** en la precisión del MLP, al menos dentro del rango analizado.
- **No se detecta interacción significativa** entre ambos factores.
- El modelo puede **beneficiarse de simplificación** (eliminando términos no significativos), o de **mayor cantidad de datos** (más réplicas o puntos del diseño) para mejorar la confiabilidad.




## Derivación del Punto Óptimo de la Superficie de Respuesta

Queremos encontrar el punto donde la precisión del MLP es máxima, usando el modelo cuadrático ajustado mediante el Método de Superficie de Respuesta (RSM). El modelo ajustado es:

$$
y(x_1, x_2) = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_{11} x_1^2 + \beta_{22} x_2^2 + \beta_{12} x_1 x_2
$$

### Modelo con coeficientes ajustados

Sustituimos los coeficientes obtenidos del ajuste del modelo (OLS):

$$
y(x_1, x_2) = 0.9714 + 0.1607 x_1 + 0.0058 x_2 - 0.1655 x_1^2 + 0.0475 x_2^2 - 0.0034 x_1 x_2
$$



### Derivadas parciales

Para encontrar el punto óptimo, derivamos parcialmente respecto a $x_1$ y $x_2$, e igualamos a cero:

$$
\frac{\partial y}{\partial x_1} = \beta_1 + 2\beta_{11} x_1 + \beta_{12} x_2 = 0
$$

$$
\frac{\partial y}{\partial x_2} = \beta_2 + 2\beta_{22} x_2 + \beta_{12} x_1 = 0
$$

Sustituimos los valores numéricos:

**Ecuación 1:**

$$
0.1607 - 2(0.1655) x_1 - 0.0034 x_2 = 0
$$

**Ecuación 2:**

$$
0.0058 + 2(0.0475) x_2 - 0.0034 x_1 = 0
$$



### Sistema de ecuaciones lineales

Simplificamos:

1. $-0.331 x_1 - 0.0034 x_2 = -0.1607$  
2. $-0.0034 x_1 + 0.095 x_2 = -0.0058$



### Solución del sistema

Resolviendo el sistema se obtiene:

- $x_1^* = 0.486$  
- $x_2^* = -0.044$

Este es el **punto óptimo en coordenadas codificadas**.


###  Conversión a valores reales

Usamos la fórmula de transformación de coordenadas codificadas a reales:

$$
x_{\text{real}} = x_{\text{center}} + x^* \cdot \frac{x_{\text{high}} - x_{\text{low}}}{2}
$$

- Para **learning rate** ($x_1$):  
  $0.001 + 0.486 \cdot \frac{0.01 - 0.0001}{2} \approx 0.0034$

- Para **units** ($x_2$):  
  $128 + (-0.044) \cdot \frac{256 - 64}{2} \approx 124$


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Cargar datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar píxeles
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

#  Learning rate óptimo
learning_rate_opt = 0.0034
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_opt)

#  Modelo MLP con 124 neuronas (óptimo)
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(124, activation='relu'),  # <-- Cambiado de 128 a 124
    Dense(10, activation='softmax')
])

# Compilar con optimizador modificado
model.compile(
    optimizer=optimizer,  # <-- Optimizer con tasa de aprendizaje óptima
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Entrenar modelo
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    verbose=2
)

# Graficar pérdida
plt.plot(history.history['loss'], label='Pérdida entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida validación')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()
plt.title('Convergencia de la función de pérdida')
plt.show()

# Evaluar en test
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

# Predicciones
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Reporte sklearn
print("\nReporte de clasificación (precision, recall, f1-score):")
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
750/750 - 6s - 7ms/step - accuracy: 0.9268 - loss: 0.2430 - val_accuracy: 0.9560 - val_loss: 0.1526
Epoch 2/10
